In [ ]:
import configparser
import datetime
import os
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import monotonically_increasing_id

In [ ]:
!pip install --upgrade setuptools
!pip install --upgrade pip

In [ ]:
!pip install python-snappy
!pip install pyarrow

In [ ]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [ ]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark
print('Creating spark session on AWS')
spark = create_spark_session()


input_data = "s3a://udacity-dend/"
song_input_data = "data/songdata/song_data/A/A/A/*.json"
log_input_data = "data/logdata/*.json"
output_data = "data/outputdata/"

In [ ]:
print('Read song data from json file')
song_data = spark.read.json(song_input_data)
    
# read song data file
print('Print song data schema')
df = song_data
print(df.count())
df.printSchema()

In [ ]:
print('Extract columns to create song table')
   
#print('Songs table: ')
songs_table = df.select("song_id", "title", "artist_id", "year", "duration")
print(songs_table.limit(5).toPandas())

In [ ]:
df_songs_table = songs_table.toPandas()
year_list = list(set(df_songs_table['year'].tolist()))
type(year_list[1])

In [ ]:
artist_id_list = list(set(df_songs_table['artist_id'].tolist()))
type(artist_id_list[1])

In [ ]:
df_songs_table.loc[(df_songs_table['year']==int(1982)) & (df_songs_table['artist_id']==str('AR7G5I41187FB4CE6C'))]

In [ ]:
# write songs table to parquet files partitioned by year and artist
print('Writing to parquet')
songs_table.write.partitionBy('year', 'artist_id').parquet(os.path.join(output_data, 'songs/songs.parquet'), 'overwrite')
print("done songs_table write")
#for year in year_list:
#    for artist_id in artist_id_list:
#        df_to_parquet = df_songs_table.loc[(df_songs_table['year']==int(year)) & (df_songs_table['artist_id']==str(artist_id))]
#        df_to_parquet.to_parquet("{}/songs_table/{}/{}/songs_table.parquet".format(output_data,year,artist_id))

In [ ]:
print('Artist table: ')
artists_table = df.select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude")
print(artists_table.limit(5).toPandas())

In [ ]:
#Process Log output data
# get filepath to log data file
log_data = spark.read.json(log_input_data)

# read log data file
print('Print song data schema')
log_df = log_data
print(df.count())
log_df.printSchema()
print(log_df.limit(5).toPandas())

In [ ]:
print('Users table: ')
users_table = log_df.select("firstName", "lastName", "gender", "level", "userId")
print(users_table.limit(5).toPandas())

In [ ]:
#create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000), TimestampType())
log_df = log_df.withColumn("timestamp", get_timestamp(log_df.ts))
log_df.printSchema()
log_df.head(1)

In [ ]:
# create datetime column from original timestamp column
get_datetime = udf(lambda x: F.to_date(x), TimestampType())
log_df = log_df.withColumn("start_time", get_timestamp(log_df.ts))
log_df.printSchema()
log_df.head(1)

In [ ]:
# extract columns to create time table
log_df = log_df.withColumn("hour", F.hour("timestamp"))
log_df = log_df.withColumn("day", F.dayofweek("timestamp"))
log_df = log_df.withColumn("week", F.weekofyear("timestamp"))
log_df = log_df.withColumn("month", F.month("timestamp"))
log_df = log_df.withColumn("year", F.year("timestamp"))
log_df = log_df.withColumn("weekday", F.dayofweek("timestamp"))
#log_df.printSchema()
print(log_df.limit(5).toPandas())

In [ ]:
time_table = log_df.select("start_time", "hour", "day", "week", "month", "year", "weekday")
time_table.createOrReplaceTempView("time_table")
print("Created temp view time_table")

In [ ]:
# create sql query to partition on year and month
time_table = spark.sql(
    """SELECT DISTINCT start_time, hour, day, week, month, year, weekday
    FROM time_table 
    ORDER BY year, month
    """)
# write time table to parquet files partitioned by year and month
print('Writing time_table to parquet')
df_time_table = time_table.toPandas()
year_list = list(set(df_time_table['year'].tolist()))
month_list = list(set(df_time_table['month'].tolist()))

time_table.write.partitionBy('year', 'month').parquet(os.path.join(output_data, 'timetable/time_table.parquet'),
                                                           'overwrite')

year_list = list(set(df_time_table['year'].tolist()))
month_list = list(set(df_time_table['month'].tolist()))
time_table.write.partitionBy('year', 'month').parquet(os.path.join(output_data, 'timetable/time_table.parquet'),
                                                           'overwrite')    

In [ ]:
# read in song data to use for songplays table
song_df = spark.read.json(song_input_data)

In [ ]:
song_df.printSchema()

In [ ]:
log_df.printSchema()

In [ ]:
log_df.createOrReplaceTempView("log_df_table")
song_df.createOrReplaceTempView("song_df_table")

In [ ]:
# extract columns from joined song and log datasets to create songplays table 
# songplay_id, 
# start_time, 
# user_id, 
# level, 
# song_id, 
# artist_id, 
# session_id, 
# location, 
# user_agent

songplays_table = spark.sql(
    """SELECT log_df_table.start_time, log_df_table.userId, log_df_table.level, log_df_table.sessionId, log_df_table.location, log_df_table.userAgent, song_df_table.song_id, song_df_table.artist_id 
    FROM log_df_table 
    INNER JOIN song_df_table 
    ON song_df_table.artist_name = log_df_table.artist 
    """)

In [ ]:
songplays_table = songplays_table.withColumn("songplay_id", monotonically_increasing_id())
songplays_table.show(5)

In [ ]:
# write songplays table to parquet files partitioned by year and month
print('Writing to parquet')
songplays_table.write.parquet("data/songplays_table.parquet",'overwrite')

In [ ]:
def main():
    spark = create_spark_session()
    input_data = "s3a://udacity-dend/"
    output_data = ""
    
    process_song_data(spark, input_data, output_data)    
    process_log_data(spark, input_data, output_data)


if __name__ == "__main__":
    main()